In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from os import path

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as ln
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from datetime import datetime, date
from dateutil.relativedelta import relativedelta



In [3]:
encounters = pd.read_csv("data/encounters.csv") 

In [4]:
#Create encounter_date for date of encounter in datetime format to a date from dateime
encounters['Encounter_Date'] = pd.to_datetime(encounters.START, format='%Y-%m-%d')
encounters['Encounter_Date'] = encounters['Encounter_Date'].dt.strftime('%Y-%m-%d')
encounters['Encounter_Date'] = pd.to_datetime(encounters['Encounter_Date'], format='%Y-%m-%d')

In [5]:
# added county of residence for each patient
patients = pd.read_csv("data/patients_county.csv",encoding='latin-1') 

In [6]:
conditions = pd.read_csv("data/conditions.csv")

In [7]:
allergies = pd.read_csv("data/allergies.csv")

In [8]:
medications = pd.read_csv("data/medications.csv")

In [9]:
observations = pd.read_csv("data/observations.csv")

In [10]:
# Convert Patients BIRTHDATE to date format and correct 2 digit year to proper century
patients['DOB'] = pd.to_datetime(patients['BIRTHDATE'], format = ("%m/%d/%y"))
patients['DOB'] = np.where(patients['DOB'].dt.year > 2010, patients['DOB']
                           - pd.offsets.DateOffset(years=100), patients['DOB'])

In [11]:
# dropped some columns not needed
patients.drop(['DRIVERS','PASSPORT','MAIDEN','SSN'], inplace=True, axis=1)

In [12]:
# rename columns
patients = patients.rename(columns={'Id': 'Patient_ID'})

In [13]:
# rename columns
conditions = conditions.rename(columns={'START': 'Cond_Start', 'STOP': 'Cond_Stop',
                                        'PATIENT': 'Patient_ID', 'CODE': 'Cond_Code', 
                                        'DESCRIPTION': 'Cond_Description', 'ENCOUNTER': 'Encounter_ID'}) 

In [14]:
# rename columns
encounters = encounters.rename(columns={'START': 'Visit_Start', 'STOP': 'Visit_Stop', 
                                        'PATIENT': 'Patient_ID','ENCOUNTERCLASS': 'Encounter_Class',
                                       'Id': 'Encounter_ID', 'CODE': 'Encounter_Code',
                                       'DESCRIPTION': 'Encounter_Description',
                                       'REASONCODE': 'Encounter_Reason_Code','COST': 'Encounter_Cost', 
                                        'REASONDESCRIPTION': 'Encounter_Reason_Desc'})

In [15]:
# rename columns
medications = medications.rename(columns={'START': 'Med_Start', 'STOP': 'Med_Stop', 
                                          'PATIENT': 'Patient_ID', 'ENCOUNTER': 'Encounter_ID', 
                                          'CODE': 'Med_Code', 'DESCRIPTION': 'Med_Description', 
                                         'COST': 'Med_Cost', 'TOTALCOST': 'Med_Total_Cost', 
                                         'REASONCODE': 'Med_Reason_Code', 'REASONDESCRIPTION': 'Med_Reason_Desc',
                                         'DISPENSES': 'Med_Dispenses'})

In [16]:
# rename columns
observations = observations.rename(columns={'DATE': 'Obs_Date', 
                                          'PATIENT': 'Patient_ID', 'ENCOUNTER': 'Encounter_ID', 
                                          'CODE': 'Obs_Code', 'DESCRIPTION': 'Obs_Description', 
                                         'VALUE': 'Obs_Value', 'UNITS': 'Obs_Units', 
                                         'TYPE': 'Obs_Type'})

In [17]:
# rename columns
allergies = allergies.rename(columns={'START': 'Allergy_Start', 'STOP': 'Allergy_Stop', 
                                          'PATIENT': 'Patient_ID', 'ENCOUNTER': 'Encounter_ID', 
                                          'CODE': 'Allergy_Code', 'DESCRIPTION': 'Allergy_Description'})

In [18]:
# Asthma related conditions we're interested in
# Other conditions are known co-morbidities for with Asthma
asthma_conditions = ['Asthma', 'Perennial allergic rhinitis',
                     'Perennial allergic rhinitis with seasonal variation',
                     'Seasonal allergic rhinitis',
                     'Chronic obstructive bronchitis (disorder)', 'Childhood asthma']

In [19]:
# Pulls in the dataframes of our conditions of interest into a new dataframe to look at
asthma_conditions_df = encounters[encounters['Encounter_Reason_Desc'].isin(asthma_conditions)]# Count of conditions in our dataframe
asthma_conditions_df['Encounter_Reason_Desc'].value_counts()

Asthma                                                 19146
Childhood asthma                                       15510
Perennial allergic rhinitis                             3782
Perennial allergic rhinitis with seasonal variation     3729
Seasonal allergic rhinitis                              1897
Chronic obstructive bronchitis (disorder)                209
Name: Encounter_Reason_Desc, dtype: int64

In [20]:
# Count of conditions in our dataframe
asthma_conditions_df['Encounter_Reason_Desc'].value_counts()

Asthma                                                 19146
Childhood asthma                                       15510
Perennial allergic rhinitis                             3782
Perennial allergic rhinitis with seasonal variation     3729
Seasonal allergic rhinitis                              1897
Chronic obstructive bronchitis (disorder)                209
Name: Encounter_Reason_Desc, dtype: int64

In [21]:
# These are some of the important co-moribidites associated with Asthma Admissions, creating flags in conditions dataframe
asthma_conditions_df['Asthma_Diag'] = np.where(asthma_conditions_df['Encounter_Reason_Desc'] == 'Asthma',1,0)
asthma_conditions_df['Childhood_Asthma_Diag'] = np.where(asthma_conditions_df['Encounter_Reason_Desc'] ==
                                                'Childhood asthma',1,0)
asthma_conditions_df['Rhinitis_Diag'] = np.where(asthma_conditions_df['Encounter_Reason_Desc'] == 
                                                 'Perennial allergic rhinitis',1,0)
asthma_conditions_df['Seasonal_Rhinitis_Diag'] = np.where(asthma_conditions_df['Encounter_Reason_Desc'] == 
                                                'Seasonal allergic rhinitis',1,0)
asthma_conditions_df['Chronic_Bronchitis_diag'] = np.where(asthma_conditions_df['Encounter_Reason_Desc'] == 
                                                'Chronic obstructive bronchitis (disorder)',1,0)

In [22]:
# Count of 
asthma_conditions_df['Encounter_Description'].value_counts()

Asthma follow-up                            24572
Encounter for symptom                       14815
Emergency hospital admission for asthma      4677
Admission to thoracic surgery department      209
Name: Encounter_Description, dtype: int64

In [23]:
# Creating a set of variables that we can use for evaluating its impact on Asthma admissions
patient_encounter_cond = asthma_conditions_df.groupby(['Patient_ID']).agg(
    {'Asthma_Diag': 'max', 'Childhood_Asthma_Diag': 'max', 'Rhinitis_Diag': 'max',
     'Seasonal_Rhinitis_Diag': 'max', 'Chronic_Bronchitis_diag': 'max'}).reset_index()

In [24]:
# Creating data where we can summraize the encounter types into these 3 new fields
asthma_conditions_df['Follow_Up_Asthma'] = np.where(asthma_conditions_df['Encounter_Description'] ==
                                                    'Asthma follow-up',1,0)
asthma_conditions_df['Symptom_Encounter'] = np.where(asthma_conditions_df['Encounter_Description'] ==
                                                    'Encounter for symptom',1,0)
asthma_conditions_df['Asthma_Hospital_Admission'] = np.where(asthma_conditions_df['Encounter_Description'] ==
                                                    'Emergency hospital admission for asthma',1,0)
asthma_conditions_df['Thoracic_Surgery_Admission'] = np.where(asthma_conditions_df['Encounter_Description'] ==
                                                    'Admission to thoracic surgery department',1,0)

In [25]:
# Creating a dataframe for Asthma Utilization we will look at later
asthma_utilizations = asthma_conditions_df.groupby('Patient_ID').agg(
    {'Follow_Up_Asthma': 'sum', 'Symptom_Encounter':'sum', 'Asthma_Hospital_Admission': 'sum', 
     'Thoracic_Surgery_Admission': 'sum'}).reset_index()

In [26]:
# Asthma Related Data pulled from the Encounters Table joined with utilization data for modeling
asthma_encounters = patient_encounter_cond.merge(asthma_utilizations, on = 'Patient_ID')

In [27]:
# Creating flags based on race for further analysis
patients['Race_White'] = np.where(patients['RACE'] == 'white',1,0)
patients['Race_Black'] = np.where(patients['RACE'] == 'black',1,0)
patients['Race_Hispanic'] = np.where(patients['RACE'] == 'hispanic',1,0)
patients['Race_Asian'] = np.where(patients['RACE'] == 'asian',1,0)
patients['Race_Native'] = np.where(patients['RACE'] == 'native',1,0)
patients['Race_Other'] = np.where(patients['RACE'] == 'other',1,0)

In [28]:
# Creating new flags based on sex for further analysis
patients['Sex_Male'] = np.where(patients['GENDER'] == 'M',1,0)

In [29]:
# Historical Air Quality for North Carolina Counties 
# https://www.airnow.gov/state/?name=north-carolina
Air_Quality = pd.read_excel('data/Historical_Air_Quality.xlsx')

# Changing data so we can match it
Air_Quality['County'] = Air_Quality['County'].astype(str).str.strip().str.upper().str.replace(
    'COUNTY', '').str.strip()

In [30]:
# Changing County so we can map it better
patients['COUNTY'] = patients['COUNTY'].astype(str).str.strip().str.upper().str.replace('COUNTY', '').str.strip()

In [31]:
patients = patients.merge(Air_Quality, how = 'left', left_on = 'COUNTY', right_on = 'County')

In [32]:
# We are filling in historical air quality with the average in the event it's not there
patients['Historical_PM25_AirQuality'].fillna(patients['Historical_PM25_AirQuality'].mean(), inplace = True )

# Replacing blank Air Quality Status with Moderate, which is the average
patients['Air_Quality_Status'] = patients['Air_Quality_Status'].astype(str).str.strip().str.replace(
    'nan', 'Moderate')

# Created Air Quality Variables
patients['Moderate_Air'] = np.where(patients['Air_Quality_Status'] == 'Moderate', 1,0)
patients['Good_Air'] = np.where(patients['Air_Quality_Status'] == 'Good', 1,0)
patients['Unstable_Air'] = np.where(patients['Air_Quality_Status'] == 'Unstable', 1,0)

In [33]:
#data obtained from https://linc.osbm.nc.gov/pages/home/
nc_poverty = pd.read_csv("data/nc_poverty_income.csv", thousands=',')

In [34]:
# Changing data type for merging
nc_poverty['COUNTY'] = nc_poverty['COUNTY'].astype(str).str.strip().str.upper().str.replace(
    'COUNTY','').str.strip()

#add variable for poverty rate - convert poverty % to rate
nc_poverty['Poverty_rate'] = nc_poverty['POVERTY_PCT']/100

#add variable for log base 10 of median household income
nc_poverty['Log_median_income'] = np.log10(nc_poverty.MEDIAN_HH_INCOME)



In [35]:
# Used this to create our cutoffs for poverty rates 
nc_poverty['Poverty_rate'].describe()

count    100.000000
mean       0.158010
std        0.047518
min        0.073000
25%        0.125000
50%        0.152000
75%        0.185250
max        0.315000
Name: Poverty_rate, dtype: float64

In [36]:
# Creating Variables based on poverty rates, using the 50% threshold for likely to be in poverty
# CDC Classified 40% and higher as extreme poverty areas, 20-40% to moderate poverty, <20% normal poverty
# Used top 75% for higher poverty in NC, lower 25% for low poverty in NC, moderate for everyone else
nc_poverty['High_Poverty'] = np.where(nc_poverty['Poverty_rate'] >= .19,1,0)
nc_poverty['Moderate_Poverty'] = np.where((nc_poverty['Poverty_rate'] >= .120) & 
                                          (nc_poverty['Poverty_rate'] < .19) ,1,0)
nc_poverty['Lower_Poverty'] = np.where((nc_poverty['Poverty_rate'] < .120) ,1,0)

In [37]:
patients = patients.merge(nc_poverty, how = 'left', on = 'COUNTY')

In [38]:
# Creating a patients dataframe for modeling purposes
patient_df = patients[['Patient_ID', 'DOB', 'Race_White',
       'Race_Black', 'Race_Hispanic', 'Race_Asian', 'Race_Native',
       'Race_Other', 'Sex_Male','High_Poverty', 'Moderate_Poverty',
       'Lower_Poverty', 'Moderate_Air', 'Good_Air', 'Unstable_Air']]

In [39]:
# our base population for the study using variables we will be modeling
asthma_df = pd.merge(asthma_encounters, patient_df, how='left', on='Patient_ID')

In [40]:
# Create a datafrmae from medications table, pulling all drugs from study based on conditions of interest
med_study = medications[medications['Med_Reason_Desc'].isin(asthma_conditions)]

# Checks to see if patient is one one of the drugs of interest
med_study['120_ACTUAT_Fluticasone_.044_Inhaler'] = np.where(
    med_study['Med_Description'] =='120 ACTUAT Fluticasone propionate 0.044 MG/ACTUAT Metered Dose Inhaler',1,0)
med_study['200_ACTUAT_Albuterol_.09_Inhaler'] = np.where(
    med_study['Med_Description'] == '200 ACTUAT Albuterol 0.09 MG/ACTUAT Metered Dose Inhaler',1,0)
med_study['60_ACTUAT_Fluticasone_.25_Inhaler'] = np.where(
    med_study['Med_Description'] == '60 ACTUAT Fluticasone propionate 0.25 MG/ACTUAT / salmeterol 0.05 MG/ACTUAT Dry Powder Inhaler',1,0)




In [41]:
# Creates a dataframe based on the drugs they're on for further study
medication_asthma = med_study.groupby(['Patient_ID']).agg(
    {'120_ACTUAT_Fluticasone_.044_Inhaler': 'max', '200_ACTUAT_Albuterol_.09_Inhaler': 'max',
     '60_ACTUAT_Fluticasone_.25_Inhaler': 'max'}).reset_index()

In [42]:
# Adding Medications into the Asthma Dataframe
asthma_df = pd.merge(asthma_df, medication_asthma, how='left', on='Patient_ID') 

In [43]:
# Drops duplicates in conditions list 
condt = conditions.drop_duplicates(['Patient_ID', 'Cond_Description'])

In [44]:
# add column environmental allergy in which 1 if present and 0 if absent
# Environment: food, animals, trees, grass, latex, bees, dust mites
allergies['Environment_allergies'] = np.where((allergies['Allergy_Description']=='Allergy to mould') |
                                              (allergies['Allergy_Description']=='Dander (animal) allergy') |
                                              (allergies['Allergy_Description']=='Allergy to tree pollen') |
                                              (allergies['Allergy_Description']=='Allergy to grass pollen') |
                                               (allergies['Allergy_Description']=='Latex allergy') |
                                              (allergies['Allergy_Description']=='Allergy to bee venom') |
                                              (allergies['Allergy_Description']=='House dust mite allergy'),1,0)

In [45]:
# add column food allergy in which 1 if present and 0 if absent
# Food: Eggs, dairy, soy, shellfish, fish, nutg, peanuts           
allergies['Food_Allergies'] = np.where((allergies['Allergy_Description']=='Allergy to wheat') |
                                              (allergies['Allergy_Description']=='Allergy to eggs') |
                                              (allergies['Allergy_Description']=='Allergy to dairy product') |
                                              (allergies['Allergy_Description']=='Allergy to soya') |
                                              (allergies['Allergy_Description']=='Shellfish allergy') |
                                              (allergies['Allergy_Description']=='Allergy to fish') |
                                              (allergies['Allergy_Description']=='Allergy to nut') |
                                              (allergies['Allergy_Description']=='Allergy to peanuts'),1,0)

In [46]:
# Creates a list of all allergies a patient has on a new column All_Allergies
allergies_list = allergies.groupby(['Patient_ID'])['Allergy_Description'].apply(
    list).reset_index(name = 'All_Allergies')

In [47]:
# Creates flags for Food and Environmental Allergies
allergy1 = allergies.groupby(['Patient_ID']).agg(
    {'Food_Allergies': 'max', 'Environment_allergies': 'max'}).reset_index()

In [48]:
# Allergy Dataframe to join on for furhter analysis
allergy_df = allergy1.merge(allergies_list, on = 'Patient_ID')

In [49]:
# our base population for the study using variables we will be modeling
asthma_df = pd.merge(asthma_encounters, patient_df, how='left', on='Patient_ID')
# Adding Medications into the Asthma Dataframe
asthma_df = pd.merge(asthma_df, medication_asthma, how='left', on='Patient_ID') 
# Adding Conditions List to Asthma dataframe
#asthma_df = pd.merge(asthma_df, Condition_List, how='left', on=['Patient_ID'])
# left join asthma_encounters with allergies
asthma_df = pd.merge(asthma_df, allergy_df, how='left', on=['Patient_ID'])

In [50]:
asthma_df['Food_Allergies'].fillna(0)

0        1.0
1        0.0
2        1.0
3        0.0
4        1.0
        ... 
12228    0.0
12229    1.0
12230    0.0
12231    1.0
12232    1.0
Name: Food_Allergies, Length: 12233, dtype: float64

In [51]:
asthma_df['Fluticasone_Drug'] = np.where((asthma_df['120_ACTUAT_Fluticasone_.044_Inhaler'] == 1)| 
                                          (asthma_df['60_ACTUAT_Fluticasone_.25_Inhaler']) ==1,1,0 )

In [52]:
asthma_df['Albuterol_Drug'] = np.where((asthma_df['200_ACTUAT_Albuterol_.09_Inhaler'] == 1),1,0 )

In [53]:
asthma_df['High_Follow-Up_Asthma_Visits'] = np.where(asthma_df['Follow_Up_Asthma']>5,1,0)

In [54]:
asthma_new = asthma_df.fillna(0)

In [55]:
asthma_new['Asthma_ED_Admission_Flag'] = np.where(asthma_new['Asthma_Hospital_Admission'] > 0, 1,0)

In [56]:
#create a dataframe for modeling
asthma_dat = asthma_new[['Asthma_ED_Admission_Flag','Rhinitis_Diag','Seasonal_Rhinitis_Diag',
                 'Chronic_Bronchitis_diag','Race_Black','Race_Hispanic','Race_Native','Sex_Male',
                  'High_Poverty','Moderate_Air','Unstable_Air','Food_Allergies','Environment_allergies',
                'Fluticasone_Drug','Albuterol_Drug','High_Follow-Up_Asthma_Visits']]

In [57]:
asthma_dat.to_csv("data/asthma_final.csv", index=False)

**Data dictionary for asthma_final_data <br>**
<br>**Response variable =**<br>
Asthma_ED_Admission_Flag<br>

**Predictors =**
<br>Rhinitis_Diag = 1 if present and 0 if not present
<br>Seasonal_Rhinitis_Diag = 1 if present and 0 if not present
<br>Chronic_Bronchitis_diag = 1 if present and 0 if not present
<br>Race_Black = 1 if Race recorded as 'black' and 0 if not 
<br>Race_Hispanic = 1 if Race recorded as 'hispanic' and 0 if not
<br>Race_Native = 1 if Race recorded as 'native' and 0 if not
<br>Sex_Male = 1 if gender recorded as male ('M') and 0 if not
<br>High_Poverty = 1 if poverty rate above 0.15 (highest 75%) and 0 if 0.15 or lower
<br>Moderate_Air = 1 if Historical_PM25_AirQuality = moderate and 0 if not
<br>Unstable_Air = 1 if Historical_PM25_AirQuality = unstable and 0 if not
<br>Food_Allergies = 1 if present and 0 if not present
<br>Environment_allergies = 1 if present and 0 if not present
<br>Fluticasone_Drug = 1 if fluticasone prescribed and 0 if not 
<br>Albuterol_Drug = 1 if albuterol prescribed and 0 if not 
<br>High_Follow-Up_Asthma_Visits = 1 if >= 5 follow-up visits for asthma, otherwise 0